In [ ]:
from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from glob import glob

manualSeed = 999

print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:


import tkinter as tk
from tkinter import messagebox

def start_gan():
    root.destroy()  

root = tk.Tk()
root.title("Start GAN")

root.configure(bg='#f0f0f0')
window_width = 400
window_height = 300
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x_coordinate = int((screen_width/2) - (window_width/2))
y_coordinate = int((screen_height/2) - (window_height/2))
root.geometry("{}x{}+{}+{}".format(window_width, window_height, x_coordinate, y_coordinate))

label_welcome = tk.Label(root, text="WELCOME TO DESKTOP ADAPTATION APP", font=("Helvetica", 16), bg='#f0f0f0')
label_welcome.pack(side="bottom", pady=10)  #

btn_start_gan = tk.Button(root, text="Start GAN", command=start_gan, width=30, height=5,
                          bg='#4CAF50', fg='white')
btn_start_gan.pack(pady=20)

root.mainloop()

In [ ]:
# Root directory for dataset
dataroot = "boat"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 16

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 150

# Size of feature maps in generator
ngf = 128

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 10

# Learning rate for optimizers
lr = 0.0005

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1



In [ ]:

# Create the dataset
boat = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(boat, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Create the generator
netG = Generator(ngpu).to(device)


if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))


netG.apply(weights_init)


print(netG)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
           
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)


if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))


netD.apply(weights_init)


print(netD)

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

fixed_noise = torch.randn(image_size, nz, 1, 1, device=device)

real_label = 1.
Created_label = 0.


optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")

for epoch in range(num_epochs):

    for i, data in enumerate(dataloader, 0):

      
        #  Update D network: maximize log(D(x)) + log(1 - D(G(z)))
       
        # Train with all-real batch
        netD.zero_grad()

        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

        output = netD(real_cpu).view(-1)

        errD_real = criterion(output, label)

        errD_real.backward()
        D_x = output.mean().item()

        # Train with all-Created batch
    
        noise = torch.randn(b_size, nz, 1, 1, device=device)

        Created = netG(noise)
        label.fill_(Created_label)

        output = netD(Created.detach()).view(-1)

        errD_Created = criterion(output, label)

        errD_Created.backward()
        D_G_z1 = output.mean().item()
        
        errD = errD_real + errD_Created
        
        optimizerD.step()

        # Update G network: maximize log(D(G(z)))
       
        netG.zero_grad()
        label.fill_(real_label)  
       
        output = netD(Created).view(-1)
        
        errG = criterion(output, label)
       
        errG.backward()
        D_G_z2 = output.mean().item()
       
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                Created = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(Created, padding=2, normalize=True))

        iters += 1

In [ ]:

real_batch = next(iter(dataloader))
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Created Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

In [ ]:
# Save Created images
if not os.path.exists('output/'):
    os.mkdir('output/')

version = len(glob("output/*/"))
os.mkdir(f'output/{version}')
print(f"Output version: {version}")

for img_no in range(Created.shape[0]):
    vutils.save_image(Created[img_no],f'output/{version}/{img_no}.png', normalize=True)


In [ ]:
#Convert
from PIL import Image
import os
from glob import glob

def convert_png_to_ico(png_folder, ico_output_folder):
    
    icon_folder = os.path.join(ico_output_folder, 'icon')
    if not os.path.exists(icon_folder):
        os.mkdir(icon_folder)

    
    png_files = glob(os.path.join(png_folder, '*.png'))

    if not png_files:
        print("No PNG files found in the specified folder.")
        return

    for png_file in png_files:
        try:
          
            img = Image.open(png_file)
            
            path, filename = os.path.split(png_file)
            name, ext = os.path.splitext(filename)

            ico_path = os.path.join(icon_folder, f'{name}.ico')

            img.save(ico_path, format='ICO')

            print(f"Converted {png_file} to {ico_path}")

        except Exception as e:
            print(f"Error converting {png_file}: {e}")


png_folder = r'C:\Users\Oguzhan\Desktop\gp1\output\1'
ico_output_folder = r'C:\Users\Oguzhan\Desktop\gp1\output'
convert_png_to_ico(png_folder, ico_output_folder)